In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd

import pandas as pd
import numpy as np

gtfstk_dir = '../../gtfstk/'
sys.path.append(gtfstk_dir)
import gtfstk as gt

%load_ext autoreload
%autoreload 2

data_dir = gtfstk_dir + 'data/'

In [5]:
from copy import copy


path = data_dir + 'cairns_gtfs.zip'
feed1 = gt.read_gtfs(path)
feed2 = copy(feed1)

feed2 = gt.create_shapes(feed1, all_trips=True)
feed1

45


,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat
29,shape_00,0,145.680668,-16.790759
31,shape_00,1,145.678743,-16.784664
33,shape_00,2,145.682661,-16.783111
35,shape_00,3,145.683072,-16.779088
37,shape_00,4,145.684733,-16.788134
40,shape_00,5,145.680737,-16.794710
42,shape_00,6,145.687685,-16.795858
44,shape_00,7,145.690999,-16.790495
46,shape_00,8,145.696766,-16.787267
50,shape_00,9,145.698902,-16.785488


In [10]:
ts = gt.compute_trips_stats(feed)
rs = gt.compute_routes_stats(feed, ts, date)
rs


,route_id,route_short_name,num_trips,is_loop,is_bidirectional,start_time,end_time,max_headway,min_headway,mean_headway,peak_num_trips,peak_start_time,peak_end_time,service_distance,service_duration,service_speed,mean_trip_distance,mean_trip_duration
0,110-423,110,59,0,1,05:50:00,24:02:00,35,23,29.955556,5,18:13:00,18:20:00,1894.223063,57.350000,33.029173,32.105476,0.972034
1,111-423,111,58,0,1,06:02:00,24:36:00,67,25,30.954545,5,07:57:00,08:05:00,2002.677623,59.350000,33.743515,34.528925,1.023276
2,112-423,112,15,1,0,07:55:00,22:31:00,60,60,60.000000,1,07:55:00,08:31:00,317.421880,9.000000,35.269098,21.161459,0.600000
3,113-423,113,6,0,1,06:05:00,18:42:00,60,60,60.000000,2,06:35:00,06:45:00,147.601565,3.933333,37.525822,24.600261,0.655556
4,120-423,120,32,0,1,05:34:00,22:23:00,60,60,60.000000,2,07:00:00,07:23:00,899.055146,26.633333,33.756764,28.095473,0.832292
5,120N-423,120N,2,0,0,22:00:00,23:51:00,NaN,NaN,NaN,1,22:00:00,22:51:00,81.004241,1.700000,47.649553,40.502120,0.850000
6,121-423,121,34,0,1,06:28:00,22:00:00,60,30,52.800000,3,07:46:00,07:48:00,586.058987,18.133333,32.319429,17.237029,0.533333
7,122-423,122,33,0,1,06:16:00,21:30:00,60,30,54.000000,2,07:16:00,07:30:00,541.875382,15.400000,35.186713,16.420466,0.466667
8,123-423,123,60,0,1,06:14:00,24:15:00,50,10,29.565217,3,07:10:00,07:31:00,1127.869577,40.466667,27.871571,18.797826,0.674444
9,130-423,130,33,0,1,06:04:00,23:01:00,60,60,60.000000,2,06:30:00,06:35:00,361.049921,17.050000,21.175948,10.940907,0.516667


In [11]:
gt.plot_headways(rs)

Exception: matplotlib has not been imported. aborting